In [1]:
import csv 
import pandas as pd
import numpy as np

CSV File Paths

In [2]:
facilities_path = '/home/elena/Documents/affirm-take-home-interview-sept-2018/large/facilities.csv'
banks_path = '/home/elena/Documents/affirm-take-home-interview-sept-2018/large/banks.csv'
covenants_path = '/home/elena/Documents/affirm-take-home-interview-sept-2018/large/covenants.csv'
loans_path = '/home/elena/Documents/affirm-take-home-interview-sept-2018/large/loans.csv'

Using Pandas to read CSV files into Dataframes

In [3]:
DF_FACILITIES = pd.read_csv(facilities_path)
DF_BANKS = pd.read_csv(banks_path)
DF_COVENANTS = pd.read_csv(covenants_path)
DF_LOANS = pd.read_csv(loans_path)

In [4]:
DF_LOANS.head().sort_values(by='default_likelihood', ascending = False)

,interest_rate,amount,id,default_likelihood,state
2,0.35,92388,3,0.07,SC
4,0.35,76811,5,0.07,WA
1,0.25,47821,2,0.05,TN
0,0.25,23549,1,0.04,LA
3,0.15,13961,4,0.01,OK


In [5]:
DF_COVENANTS.sort_values(by=['max_default_likelihood'])

,facility_id,max_default_likelihood,bank_id,banned_state
13,15,0.06,2,UT
22,6,0.08,5,KY
0,1,0.09,3,ND
1,5,0.09,1,LA
2,4,0.09,4,PA
3,7,0.09,5,RI
5,8,0.09,1,OK
7,11,0.09,3,TN
11,3,0.09,4,MA
20,2,0.09,4,TX


In [6]:
for attribute in DF_COVENANTS.columns:
    print('FEATURE:', attribute, ' UNIQUE VALUES:', len(DF_COVENANTS[attribute].unique()))

FEATURE: facility_id  UNIQUE VALUES: 15
FEATURE: max_default_likelihood  UNIQUE VALUES: 5
FEATURE: bank_id  UNIQUE VALUES: 5
FEATURE: banned_state  UNIQUE VALUES: 28


In [21]:
def clean_facility_data(DF_FACILITIES):
    DF_CLEAN_FACILITIES = DF_FACILITIES.rename(columns = {'id': 'facility_id'}).sort_values(by='facility_id').reset_index(drop=True)
    DF_CLEAN_FACILITIES.index = DF_CLEAN_FACILITIES.index + 1
    return DF_CLEAN_FACILITIES.to_dict(orient = 'index')

      amount  interest_rate  facility_id  bank_id
1  1936153.0           0.02            1        3
2  1642900.0           0.03            2        4
3  2252606.0           0.03            3        4
4  1983921.0           0.03            4        4
5  1779071.0           0.04            5        1
{1: {'amount': 1936153.0, 'interest_rate': 0.02, 'facility_id': 1, 'bank_id': 3}, 2: {'amount': 1642900.0, 'interest_rate': 0.03, 'facility_id': 2, 'bank_id': 4}, 3: {'amount': 2252606.0, 'interest_rate': 0.03, 'facility_id': 3, 'bank_id': 4}, 4: {'amount': 1983921.0, 'interest_rate': 0.03, 'facility_id': 4, 'bank_id': 4}, 5: {'amount': 1779071.0, 'interest_rate': 0.04, 'facility_id': 5, 'bank_id': 1}, 6: {'amount': 1830954.0, 'interest_rate': 0.04, 'facility_id': 6, 'bank_id': 5}, 7: {'amount': 1678565.0, 'interest_rate': 0.05, 'facility_id': 7, 'bank_id': 5}, 8: {'amount': 2234064.0, 'interest_rate': 0.05, 'facility_id': 8, 'bank_id': 1}, 9: {'amount': 2127767.0, 'interest_rate': 0.06, 'fa

In [71]:
def create_assignment_table(LOAN_DATA, COVENANT_DATA, FACILITY_DATA):
    COVENANT_DATA = COVENANT_DATA.fillna(1).sort_values(by=['bank_id'])
    DF_ASSIGNMENTS =  pd.DataFrame(columns=['loan_id', 'facility_id'])
    FACILITY_DATA = clean_facility_data(FACILITY_DATA)
    for index, loan in LOAN_DATA.iterrows():
        for index, covenant in COVENANT_DATA.iterrows():
            if (loan['default_likelihood'] < covenant['max_default_likelihood']) & (loan['state'] != covenant['banned_state']):
                facility_status = FACILITY_DATA.get(covenant['facility_id'])
                difference = facility_status.get('amount') - loan['amount']
                if (difference) > 0:
                    facility_status['amount'] = difference
                    FACILITY_DATA[covenant['facility_id']] = facility_status
                    print(covenant['facility_id'], 1111111111111, FACILITY_DATA[covenant['facility_id']])
                    DF_ASSIGNMENTS = DF_ASSIGNMENTS.append({'loan_id': loan['id'], 'facility_id':covenant['facility_id']}, ignore_index=True)
                    break
    return(DF_ASSIGNMENTS)

In [72]:
df_test = create_assignment_table(DF_LOANS, DF_COVENANTS, DF_FACILITIES)
df_test.to_csv(path_or_buf = 'assignments_test.csv', index=False)

      amount  interest_rate  facility_id  bank_id
1  1936153.0           0.02            1        3
2  1642900.0           0.03            2        4
3  2252606.0           0.03            3        4
4  1983921.0           0.03            4        4
5  1779071.0           0.04            5        1
5 1111111111111 {'amount': 1755522.0, 'interest_rate': 0.04, 'facility_id': 5, 'bank_id': 1}
5 1111111111111 {'amount': 1707701.0, 'interest_rate': 0.04, 'facility_id': 5, 'bank_id': 1}
5 1111111111111 {'amount': 1615313.0, 'interest_rate': 0.04, 'facility_id': 5, 'bank_id': 1}
14 1111111111111 {'amount': 9588.0, 'interest_rate': 0.08, 'facility_id': 14, 'bank_id': 1}
5 1111111111111 {'amount': 1538502.0, 'interest_rate': 0.04, 'facility_id': 5, 'bank_id': 1}
5 1111111111111 {'amount': 1493531.0, 'interest_rate': 0.04, 'facility_id': 5, 'bank_id': 1}
13 1111111111111 {'amount': 401911.0, 'interest_rate': 0.07, 'facility_id': 13, 'bank_id': 1}
5 1111111111111 {'amount': 1461499.0, 'interest_r

In [73]:
def get_expected_yeild(default_likelihood, loan_interest_rate, amount, facility_interest_rate):
    return (1 - default_likelihood) * loan_interest_rate * amount - default_likelihood * amount - facility_interest_rate * amount

In [74]:
def create_yeild_table(assigned_csv, LOAN_DATA, FACILITY_DATA):
    DF_ASSIGNED = pd.read_csv(assigned_csv)
    LOAN_DATA = LOAN_DATA.rename(columns = {'id':'loan_id', 'interest_rate':'loan_interest_rate'})
    FACILITY_DATA = FACILITY_DATA.rename(columns = {'id':'facility_id', 'amount':'facility_amount'})
    ASSIGNED_LOANS = pd.merge(DF_ASSIGNED, LOAN_DATA, on='loan_id')
    ALL_TOGETHER_NOW = pd.merge(FACILITY_DATA, ASSIGNED_LOANS, on = 'facility_id')
    DF_YIELD =  pd.DataFrame(columns=['facility_id', 'expected_yield'])
    for index, assignment in ALL_TOGETHER_NOW.iterrows():
        facility_amount, interest_rate, facility_id, bank_id,loan_id,loan_interest_rate, amount, default_likelihood, state = assignment
        expected_yield = get_expected_yeild(default_likelihood, loan_interest_rate, amount, interest_rate)
        DF_YIELD = DF_YIELD.append({'facility_id': facility_id, 'expected_yield':expected_yield}, ignore_index=True)
    return DF_YIELD

In [75]:
yield_test = create_yeild_table('assignments_test.csv', DF_LOANS, DF_FACILITIES)

In [76]:
yield_test = yield_test.groupby(['facility_id']).sum()

In [77]:
yield_test.to_csv(path_or_buf = 'yield_test.csv')